In [1]:
!git clone https://github.com/zabique/Wav2Lip

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/kaggle/working/Wav2Lip/checkpoints/wav2lip_gan.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd /kaggle/working/Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/kaggle/working/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install -q youtube-dl
!pip install ffmpeg-python
!pip install librosa==0.9.1

#this code for recording audio
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

from IPython.display import clear_output 
clear_output()
print("\nDone")


Done


In [4]:
!pip install gTTS

  Obtaining dependency information for gTTS from https://files.pythonhosted.org/packages/6a/4f/b133719e7638ca68f8805dd75731371db8d5ed23be84d6fc30845a46bedb/gTTS-2.5.0-py3-none-any.whl.metadata


In [5]:
from gtts import gTTS

text = "Virtual humans have become increasingly popular in various industries, such as entertainment and e-commerce."

tts_output = gTTS(text)
tts_output.save('simple_gtts.wav')

In [6]:
#@title 2.Create Wav2Lip video (using wav2lip_gan.pth) GAN
!cd /kaggle/working/Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/kaggle/input/wav2lip-dataset/MonnaLisa.mp4" --audio "/kaggle/working/simple_gtts.wav" --outfile "results/result_voice_molalisa.mp4"

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Using cuda for inference.
Reading video frames...
Number of frames available for inference: 786
/kaggle/working/Wav2Lip/audio.py:100: FutureWarning: Pass sr=16000, n_fft=800 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.filters.mel(hp.sample_rate, hp.n_fft, n_mels=hp.num_mels,
(80, 632)
Length of mel chunks: 464
  0%|                                                    | 0/29 [03:31<?, ?it/s]
Recovering from OOM error; New batch size: 8

100%|███████████████████████████████████████████| 58/58 [01:25<00:00,  1.47s/it]
Load checkpoint from: checkpoints/wav2lip_gan.pth
Model loaded
100%|█████████████████████████████████████████████| 4/4 [05:31<00:00, 82.83s/it]
ffmpeg version 4

2.Use resize_factor to reduce the video resolution, as there is a change you might get better results for lower resolution videos. Why? Because the model was trained on low resolution faces.

In [7]:
!ls /kaggle/working/Wav2Lip/results

README.md  result_voice_molalisa.mp4


In [8]:
#@title 3.Play result video -  50% scaling
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/kaggle/working/Wav2Lip/results/result_voice_molalisa.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

**USE VITS**

In [9]:
#https://huggingface.co/docs/transformers/model_doc/vits

import torch
from transformers import VitsTokenizer, VitsModel, set_seed

tokenizer = VitsTokenizer.from_pretrained("facebook/mms-tts-eng")
model = VitsModel.from_pretrained("facebook/mms-tts-eng")

inputs = tokenizer(text=text, return_tensors="pt")

set_seed(555)  # make deterministic

with torch.no_grad():
    outputs = model(**inputs)

waveform = outputs.waveform[0]
print(waveform.shape)

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/413 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/47.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/145M [00:00<?, ?B/s]

torch.Size([121344])


In [10]:
from IPython.display import Audio

Audio(waveform, rate=model.config.sampling_rate)

In [11]:
# save result
import scipy

scipy.io.wavfile.write("simple_vits_tts.wav", rate=model.config.sampling_rate, data=waveform.cpu().numpy())

In [12]:
#@title 2.Create Wav2Lip video (using wav2lip_gan.pth) GAN
!cd /kaggle/working/Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/kaggle/input/wav2lip-dataset/MonnaLisa.mp4" --audio "/kaggle/working/simple_vits_tts.wav" --outfile "results/result_voice_molalisa_v2.mp4"

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Using cuda for inference.
Reading video frames...
Number of frames available for inference: 786
/kaggle/working/Wav2Lip/audio.py:100: FutureWarning: Pass sr=16000, n_fft=800 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.filters.mel(hp.sample_rate, hp.n_fft, n_mels=hp.num_mels,
(80, 607)
Length of mel chunks: 445
  0%|                                                    | 0/28 [03:30<?, ?it/s]
Recovering from OOM error; New batch size: 8

100%|███████████████████████████████████████████| 56/56 [01:36<00:00,  1.73s/it]
Load checkpoint from: checkpoints/wav2lip_gan.pth
Model loaded
100%|█████████████████████████████████████████████| 4/4 [05:39<00:00, 84.85s/it]
ffmpeg version 4

In [13]:
#@title 3.Play result video -  50% scaling
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/kaggle/working/Wav2Lip/results/result_voice_molalisa_v2.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

3.Use more padding to include the chin region (u can manually edit pads dimensions viewing and changing the code)